In [1]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [2]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [3]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [4]:
today = datetime.datetime.today().date() # or datetime.now to use local timezone
daystart=pd.Timestamp(year=today.year, month=today.month, day=today.day, hour=0,tz='America/New_York').isoformat()

In [5]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        
        dataframe['emaclose'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['emapulse'] = ta.RSI(dataframe['emaclose'], timeperiod=3)
        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.buying_power

ACTIVE


'37591.374'

In [12]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D

In [22]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-06-10'

In [38]:
datetime.time()

datetime.time(0, 0)

In [39]:
time.localtime().tm_sec

6

In [ ]:
#Limit sell all within a 
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currentsecond=time.localtime().tm_sec
    currenttime=currenthour*100+currentminute
    currenttimestamp=api.get_clock().timestamp
    
    if currenttime ==6:29 and currentsecond >=30: 
        try:
            assetlist=api.list_positions()
        except:
            print('http error')
        for asset in assetlist:
                #if asset is in list of positions AND is more than 2.5% loss
                # sell and turn that symbol to null
            try:

                symbol=asset.symbol

                returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df

                #openPo/sition = api.get_position(asset.symbol)
                #asksize,bidsize,avgbidprice,avgaskprice=quotelister(quotes)
                returned = api.submit_order(symbol,int(asset.qty),"sell","limit","day",limit_price=returned_data['close'][-1]-.04) # Market order to fully close position
                print('Sell '+symbol)
                trade=False
            except:
                print('error')

    time.sleep(1)



C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\Jeffrey Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

Sell LBTYA
Sell FOXA
Sell UAL


In [ ]:
api.close_all_positions()

In [ ]:
goodsells